In [1]:
%pip install opencv-python


  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached numpy-2.2.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached numpy-2.2.1-cp312-cp312-win_amd64.whl (12.6 MB)


In [2]:
import cv2
import os
import json

In [4]:
video_path = "./assets/sample.mp4"
save_dir = "extracted_segments"
os.makedirs(save_dir, exist_ok=True)

In [5]:
exercise_name = "bench_press"
os.makedirs(os.path.join(save_dir, exercise_name), exist_ok=True)

In [6]:
# Tạo folder tên dạng 20250113_162600
import datetime


def get_current_time():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# Tạo tên folder sẽ lưu từng segment
dest_folder = os.path.join(save_dir, exercise_name, get_current_time())
os.makedirs(dest_folder, exist_ok=True)


cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file.")

# Thông tin video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"H264")

start_frame = None
end_frame = None
segment_count = 0
current_writer = None

# Danh sách metadata để lưu thông tin các segment
metadata = []

display_width = 400  # Kích thước hiển thị video
display_height = int(height * (display_width / width))

# Đọc và hiển thị khung hình đầu tiên
ret, first_frame = cap.read()
if ret:
    resized_frame = cv2.resize(first_frame, (display_width, display_height))
    cv2.imshow("Video", resized_frame)
    print("Press any to start playing the video...")
    cv2.waitKey(0)
else:
    print("Error: Cannot read the first frame.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Thay đổi kích thước hiển thị video
    resized_frame = cv2.resize(frame, (display_width, display_height))

    # Hiển thị video
    cv2.imshow("Video", resized_frame)

    key = cv2.waitKey(1) & 0xFF

    # Nhấn 's' để bắt đầu đoạn video
    if key == ord("s"):
        # Tạo file writer cho đoạn mới
        if current_writer is None:
            start_frame = current_frame
            print(f"Start frame: {start_frame}")

            segment_path = os.path.join(dest_folder, f"segment_{segment_count}.mp4")
            current_writer = cv2.VideoWriter(segment_path, fourcc, fps, (width, height))

    # Nhấn 'e' để kết thúc đoạn video
    elif key == ord("e") and start_frame is not None:
        end_frame = current_frame
        print(f"End frame: {end_frame}")

        # Lưu thông tin segment vào metadata
        metadata.append(
            {
                "segment_path": segment_path,
                "video_src_path": video_path,
                "start_frame": start_frame,
                "end_frame": end_frame,
                "start_time_seconds": round(start_frame / fps, 2),
                "end_time_seconds": round(end_frame / fps, 2),
            }
        )

        # Kết thúc đoạn và reset writer
        segment_count += 1
        start_frame = None
        current_writer.release()
        current_writer = None

    # Ghi khung hình vào đoạn video
    if current_writer is not None:
        current_writer.write(frame)

    # Nhấn 'q' để thoát
    if key == ord("q"):
        break

cap.release()
if current_writer:
    current_writer.release()
cv2.destroyAllWindows()

# Ghi metadata ra file JSON
metadata_path = os.path.join(dest_folder, "metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Metadata saved to {metadata_path}")

Press any to start playing the video...
Start frame: 59
End frame: 100
Start frame: 113
End frame: 187
Start frame: 199
End frame: 210
Start frame: 232
End frame: 240
Metadata saved to extracted_segments\bench_press\20250113_170239\metadata.json
